<a href="https://colab.research.google.com/github/SebastianAblerKratkey/Miscellaneous/blob/main/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy_financial
!pip install mplfinance
!pip install adjustText
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 879.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [2]:
import numpy as np
import numpy_financial as npf
import pandas as pd
from pandas_datareader.data import DataReader as dr
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
from matplotlib import gridspec
import mplfinance as mpf
import seaborn as sns
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.stats import norm
import os
import datetime
import statsmodels.api as sm
import base64
from io import StringIO, BytesIO

# this module is utilized to prevent the annotations in the plot from overlapping
from adjustText import adjust_text

# Get Yahoo Finance Data
import yfinance as yf

# Library for Website creation
import streamlit as st
st.set_option('deprecation.showPyplotGlobalUse', False)

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

def convert_date_index(df):
    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)
    # Extract the month and year from the datetime
    df.index = df.index.strftime("%b %Y")
    return df

def create_performance_index(price_df):
    returns = price_df.pct_change()
    growth = returns+1
    growth = growth.fillna(1) # set starting value for index
    index = growth.cumprod()
    index = index - 1 # deduct starting value to get the percentage change
    return index

def visualize_performance(prices, list_of_names):
    benchmarking_data = create_performance_index(prices)

    color_list = ['deepskyblue', 'steelblue', 'mediumslateblue', 'cornflowerblue', 'lightsteelblue',
                    'mediumslateblue', 'lightblue']

    benchmarking_data_filtered = benchmarking_data.filter(list_of_names)
    if len(list_of_names) > 0:
        benchmarking_data_filtered.plot(figsize=(15, 10), color=color_list)
    else:
        plt.figure(figsize=(15, 10))


    plt.fill_between(benchmarking_data.index, benchmarking_data.max(axis=1), benchmarking_data.min(axis=1),
                        color='grey', alpha=0.17, label="Range")

    # Calculate the number of days to add
    num_days = (benchmarking_data_filtered.index.max() - benchmarking_data_filtered.index.min()).days
    days_to_add1 = num_days / 120
    #days_to_add2 = num_days / 12
    days_to_add2 = 0

    # Plot scatter points at the end of each line
    for col in benchmarking_data_filtered.columns:
        #plt.scatter(benchmarking_data_filtered.index[-1], benchmarking_data_filtered[col].iloc[-1], color=color_list[list_of_names.index(col)], zorder=5)
        #text lablel is offset by a number of days to the right
        plt.text(benchmarking_data_filtered.index[-1] + pd.Timedelta(days=days_to_add1), benchmarking_data_filtered[col].iloc[-1], str(round(benchmarking_data_filtered[col].iloc[-1]*100, 2))+"%",color=color_list[list_of_names.index(col)], size=12, verticalalignment='center')

    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    plt.gca().xaxis.set_major_locator(MaxNLocator())
    plt.gca().set_xlim(left=benchmarking_data.head(1).index.max())



    plt.xlim(right=benchmarking_data.index.max() + pd.Timedelta(days=days_to_add2))  # Extend x-axis limit by number of days
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))  # Format dates to show month and year
    plt.grid('on', ls="--")
    plt.ylabel(f"Performance (indexed: {benchmarking_data.head(1).index.max().strftime('%d.%m.%Y')} = 0%)", fontsize=12)
    plt.legend(fontsize=12)

    # Rotate x-axis labels to be horizontal
    plt.xticks(rotation=0, ha='center')

    # Remove x-axis label
    plt.gca().set_xlabel('')
    plt.minorticks_off()
    plt.show()

def visualize_summary(summary):
    fontsize=8
    plt.rc('font', size=fontsize)
    fig, (ax1, ax2) = plt.subplots(1, 2, clip_on=False)
    ax1.grid('on', ls="--")
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(False)
    ax2.grid('on', ls="--")
    ax2.set_axisbelow(True)
    ax2.yaxis.grid(False)
    ax3 =  ax2.twiny()
    ax4 = ax1.twiny()
    ax1.xaxis.set_major_locator(MaxNLocator(nbins="auto"))
    ax2.xaxis.set_major_locator(MaxNLocator(nbins="auto"))
    ax3.xaxis.set_major_locator(MaxNLocator(nbins="auto"))
    ax4.xaxis.set_major_locator(MaxNLocator(prune='upper', nbins="auto"))
    x_dim = max(max(summary["mean return"]), max(summary['standard deviation'])) * 1.1
    height_of_fig = len(summary)*0.1
    ax1.set_position([0, 0, 0.35, height_of_fig])
    ax2.set_position([0.35, 0, 0.35, height_of_fig])
    ax1.set_xlim(left=0, right=x_dim)
    ax2.set_xlim(left=0, right=x_dim)
    ax3.set_xlim(left=0, right=x_dim)
    ax4.set_xlim(left=-x_dim, right=0)
    ax1.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax2.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax3.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax4.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax1.invert_xaxis()
    ax2.tick_params(left = False, bottom=False)
    ax2.set_yticklabels([])
    ax2.set_xticklabels([])
    summary_sorted = summary.copy()
    summary_sorted.sort_values("Sharpe ratio", inplace=True)
    bar_width = 0.6  # Set a fixed width for the horizontal bars
    for index, row in summary_sorted.iterrows():
        ax1.barh(index, row['standard deviation'], height=bar_width, color="steelblue")
        ax2.barh(index,  row['mean return'], height=bar_width, color="deepskyblue")
        if row['mean return'] < 0:
            if abs(row['mean return']) > abs(row['standard deviation']):
                ax1.barh(index, abs(row['mean return']), height=bar_width, color="deepskyblue")
                ax1.barh(index, row['standard deviation'], height=bar_width, color="steelblue")
            if abs(row['mean return']) <= abs(row['standard deviation']):
                ax1.barh(index, row['standard deviation'], height=bar_width, color="steelblue")
                ax1.barh(index, abs(row['mean return']), height=bar_width, color="deepskyblue")
    ax1_patch = mpatches.Patch(color='deepskyblue', label='Mean return')
    ax1.legend(handles=[ax1_patch], fontsize=fontsize, frameon=False, loc='center', ncol=2, bbox_to_anchor=(1, 1+0.8/len(summary)))
    ax2_patch = mpatches.Patch(color='steelblue', label='Volatility')
    ax2.legend(handles=[ax2_patch], fontsize=fontsize, frameon=False, loc='center', ncol=2, bbox_to_anchor=(0, -0.8/len(summary)))
    plt.show()


def visualize_correlation(corr):
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["deepskyblue", "mediumslateblue", "slategrey"])
    mask = np.triu(corr, k=1)
    plt.figure(figsize=(12, 7))
    sns.heatmap(corr, annot=True, cmap=cmap, mask=mask, linewidths=5,
                annot_kws={'color':'white'})
    plt.show()

def portfolio_std(weights):
    portfolio_std = np.sum(weights * np.sum(weights * annualized_cov_returns, axis=1)) ** 0.5
    return portfolio_std

def portfolio_return(weights, returns):
    portfolio_return = np.sum(weights * returns)
    return portfolio_return

def negative_portfolio_SR(weights, rf, returns):
    return_p = portfolio_return(weights, returns)
    std_p = portfolio_std(weights)
    negative_sharpe_ratio = -1*(return_p - rf) / std_p
    return negative_sharpe_ratio

def negative_portfolio_utility(weights, returns):
    return_p = portfolio_return(weights, returns)
    std_p = portfolio_std(weights)
    negative_portfolio_utility = -1*(return_p - 0.5*A*std_p**2)
    return negative_portfolio_utility

def create_KPI_report(name, weights, rf, returns):
    KPIs = pd.DataFrame(index=[name])
    KPIs["portfolio return"] = portfolio_return(weights, returns)
    KPIs["protfolio std"] = portfolio_std(weights)
    KPIs["sharpe ratio"] = (KPIs["portfolio return"]- rf) / KPIs["protfolio std"]
    KPIs["utility"] = KPIs["portfolio return"] - 0.5*A*KPIs["protfolio std"]**2
    return KPIs

def create_portfolio_visual(name, summary, KPIs):
    plt.figure(figsize=(8, 8))
    plt.pie(summary["weight"], wedgeprops=dict(width=0.45),
            colors=['deepskyblue', 'steelblue', 'lightblue', 'lightsteelblue', 'cornflowerblue',
                    'mediumslateblue','thistle', 'dodgerblue', 'slategrey'],
            autopct='%.2f%%',pctdistance=0.8, startangle=90,labels=summary.index)
    plt.annotate(name, xy=(0,0), fontsize=30, va="center", ha="center")
    plt.annotate("E(r): {}%".format(float((KPIs["portfolio return"]*100).round(decimals=2))),
                 xy=(-0.07,-0.18), fontsize=10, va="center", ha="right")
    plt.annotate("Vola: {}%".format(float((KPIs["protfolio std"]*100).round(decimals=2))),
                 xy=(+0.07,-0.18), fontsize=10, va="center", ha="left")
    plt.show()

def create_mvf_cal_visual():
    #plot minimum varriance frontier and CAL
    color1 = 'cornflowerblue'
    color2 = 'darkmagenta'

    plt.figure(figsize=(15, 10))

    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    plt.gca().set_xlim(left=0)
    plt.gca().set_xlim(right=max(max(summary["standard deviation"]),float(KPIs_ocp["protfolio std"]))*1.05)

    plt.scatter(summary["standard deviation"], summary["mean return"], color=color1)

    # capital allocation line

    # between std = 0 and std = std_orp_l
    std_cal_1 = np.arange(0, float(KPIs_orp_l["protfolio std"]), step)
    return_cal_1 = rf_l + float(KPIs_orp_l["sharpe ratio"])*std_cal_1
    plt.plot(std_cal_1 ,return_cal_1, color=color1, label='Capital allocation line')

    # between std_orp_l and std_orp_b -> follows minimum varriance frontier
    mvf_plot_slice = mvf_plot_data[(mvf_plot_data["return"] >= float(KPIs_orp_l["portfolio return"])) &
                               (mvf_plot_data["return"] <= float(KPIs_orp_b["portfolio return"]))]
    std_cal_2 = mvf_plot_slice["std"]
    return_cal_2 = mvf_plot_slice["return"]
    plt.plot(std_cal_2,return_cal_2,color=color1)

    # after std_orp_b
    endpoint_cal = plt.gca().get_xlim()[1]
    std_cal_3 = np.arange(float(KPIs_orp_b["protfolio std"]), endpoint_cal, step)
    return_cal_3 = rf_b + float(KPIs_orp_b["sharpe ratio"])*std_cal_3
    plt.plot(std_cal_3 ,return_cal_3, color=color1)

    # minimum varriance frontier
    plt.plot(mvf_plot_data["std"], mvf_plot_data["return"], color=color1, linestyle='--',
         label='Minimum varriance frontier')

    plt.scatter(KPIs_mvp["protfolio std"], KPIs_mvp["portfolio return"], color=color2)
    plt.scatter(KPIs_orp["protfolio std"], KPIs_orp["portfolio return"], color=color2)
    plt.scatter(KPIs_ocp["protfolio std"], KPIs_ocp["portfolio return"], color=color2)

    plt.legend(fontsize=12)
    plt.xlabel("Volatility", fontsize=12)
    plt.ylabel("Expected return", fontsize=12)
    plt.grid('on', ls="--")

    # labeling
    x_offset = plt.gca().get_xlim()[1]*0.01
    for i in summary_p.index:
        plt.annotate(i,(summary_p["protfolio std"][i], summary_p["portfolio return"][i]),
                 (summary_p["protfolio std"][i]-x_offset, summary_p["portfolio return"][i]),
                 color=color2, fontsize=12, ha='right')

    labels = []
    for i in summary.index:
        labels.append(plt.text(summary["standard deviation"][i], summary["mean return"][i], i, size=8))
    adjust_text(labels)

    plt.show()

def currency_formatter_alt_EUR_decimal_seperator(x, currency="EUR"):
         if currency == 'EUR':
            return f'{currency} {x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", ".")
         elif currency == 'USD':
            return f'{currency} {x:,.2f}'

def currency_formatter(x, currency="EUR"):
    return f'{currency} {x:,.2f}'

def currency_formatter_signs(x, currency="EUR"):
         if currency == 'EUR':
            return f'€ {x:,.2f}'
         elif currency == 'USD':
            return f'$ {x:,.2f}'

def visualize_simulaiton(sim_avg, deposits, currency='EUR'):
    """
    Plots the average simulated performance over time.

    Parameters:
    sim_avg (DataFrame): A DataFrame containing the average simulated performance.
    currency (str): The currency in which to display the performance data.

    Returns:
    None
    """
    def currency_formatter(x, pos):
         if currency == 'EUR':
            return f'€ {x:,.2f}'
         elif currency == 'USD':
            return f'$ {x:,.2f}'

    plt.figure(figsize=(15, 5))

    # Set the y-axis formatter
    plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))

    # Set the tick locations and labels
    plt.xticks(sim_avg.index)
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter('{:.0f}'.format))

    # Plot the bars
    if sim_avg.iloc[-1] > deposits[-1]:
        plt.bar(sim_avg.index, sim_avg, color='deepskyblue', label="Capital", align='center')
        plt.bar(sim_avg.index, deposits, color='steelblue', label="Money invested", align='center')
    if sim_avg.iloc[-1] < deposits[-1]:
        plt.bar(sim_avg.index, deposits, color='steelblue', label="Money invested", align='center')
        plt.bar(sim_avg.index, sim_avg, color='deepskyblue', label="Capital", align='center')

    # Set the x-axis limits based on the minimum and maximum values in the index
    plt.gca().set_xlim(left=sim_avg.index.min()-0.8)
    plt.gca().set_xlim(right=sim_avg.index.max()+0.8)

    # Rotate xticks if needed
    if len(sim_avg.index) > 22:
        plt.xticks(rotation=45)
    if len(sim_avg.index) > 60:
        plt.xticks(rotation=90)
    if len(sim_avg.index) > 75:
        plt.gca().tick_params(axis='x', labelsize=8)

    plt.gca().set_axisbelow(True)
    plt.grid('on', ls="--")
    plt.gca().xaxis.grid(False)
    plt.legend(fontsize=12)

    plt.show()

def run_and_display_monte_carlo_sim():
    # Generate the random trials
    sim_list = []
    for index, row in sim_summary.iterrows():
        sim_list.append(np.random.normal(row["mean return"], row["standard deviation"], size=(num_trials, num_months)) * row["weight"])

    simulated_returns = np.array(sim_list).sum(axis=0)

    # Calculate the potential future values of the investment
    val_future = np.zeros((num_trials, num_months+1))
    val_future[:,0] = val_today

    for i in range(1, num_months+1):
        val_future[:,i] = val_future[:,i-1] * np.exp(simulated_returns[:,i-1]) + additional_investment_per_month

    simulated_performance = pd.DataFrame(val_future).transpose()
    simulated_performance_annual = simulated_performance[simulated_performance.index % 12 == 0]


    # Set the index of the DataFrame to a range of years representing the time horizon of the simulation
    future_years = np.arange(current_year, current_year + num_years + 1, 1)
    simulated_performance_annual.set_index(pd.Index(future_years), inplace=True)

    # Calculate the average simulated performance across all trials
    avg_simulated_performance = simulated_performance_annual.mean(axis=1)

    # Calculate the cumulative deposits
    if additional_investment_per_year > 0:
        cumulative_depositis = np.arange(val_today,
                            val_today+total_additional_investments+additional_investment_per_year,
                            additional_investment_per_year)
    else:
        cumulative_depositis = np.array([val_today]*(num_years+1))

    visualize_simulaiton(avg_simulated_performance, cumulative_depositis, currency=currency)
    st.pyplot()

    expected_capital = float(avg_simulated_performance.iloc[-1])

    # Cash flow of the savings plan
    cash_flow = [-val_today]+[-additional_investment_per_month]*(num_months-1)+[avg_simulated_performance.iloc[-1]-additional_investment_per_month]
    # IRR / money weighted retun
    irr_monthly = npf.irr(cash_flow)
    irr_annual = (1 + irr_monthly)**12 - 1
    # Time weighted return
    avg_log_TWR = simulated_returns.mean(axis=0).sum() / num_years


    sim_text = (
            f"Based on each asset's historic mean return and standard deviation and their respective weight in "
            f"the {selected_p} the simulation predicts the investor's capital to reach **{currency_formatter(expected_capital, currency=currency)}** "
            f"in {current_year+num_years} based on the above specified savings plan. "
            f"This corresponds to an expected time-weighted return of {avg_log_TWR:.2%} p.a. "
            f"and an expected money-weighted return (IRR) of {irr_annual:.2%} p.a."
            )
    st.write(sim_text)
    sim_outcomes = simulated_performance.iloc[-1,:]

    percent_in_interval = 0.0

    moe = 1.0
    while percent_in_interval <= 1.0:
        in_interval = (expected_capital - moe < sim_outcomes) & (sim_outcomes < expected_capital + moe)
        percent_in_interval = in_interval.mean()
        if round(percent_in_interval,4) == p:
            break
        if percent_in_interval > p:
            moe = moe*0.9999
        elif percent_in_interval > p-0.05:
            moe = moe*1.001
        else:
            moe = moe*1.5

    p_above_mean = (expected_capital < sim_outcomes).mean()
    p_below_mean = (expected_capital > sim_outcomes).mean()

    lower_bound = expected_capital - moe
    upper_bound = expected_capital + moe

    if currency == "USD":
        st.markdown(
                    f"""
                    It is important to note that {currency_formatter(expected_capital, currency=currency)} is just the simulation mean and not a guaranteed outcome:
                    - {percent_in_interval:.0%} of simulation outcomes lie between {currency_formatter(lower_bound, currency=currency)} and {currency_formatter(upper_bound, currency=currency)}
                    - {p_below_mean:.0%} of simulation outcomes lie between {currency_formatter(expected_capital, currency=currency)} and {currency_formatter(sim_outcomes.min(), currency=currency)} (the lowest simulation outcome)
                    - {p_above_mean:.0%} of simulation outcomes lie between {currency_formatter(expected_capital, currency=currency)} and {currency_formatter(sim_outcomes.max(), currency=currency)} (the highest simulation outcome)
                    """
                    )
    else:
        st.markdown(
                    f"""
                    It is important to note that {currency_formatter(expected_capital, currency=currency)} is just the simulation mean and not a guaranteed outcome:
                    - {percent_in_interval:.0%} of simulation outcomes lie between {currency_formatter(lower_bound, currency=currency)} and {currency_formatter(upper_bound, currency=currency)}
                    - {p_below_mean:.0%} of simulation outcomes lie between {currency_formatter(expected_capital, currency=currency)} and {currency_formatter(sim_outcomes.min(), currency=currency)} (the lowest simulation outcome)
                    - {p_above_mean:.0%} of simulation outcomes lie between {currency_formatter(expected_capital, currency=currency)} and {currency_formatter(sim_outcomes.max(), currency=currency)} (the highest simulation outcome)
                    """
                    )

def generate_excel_download_link(df):
    towrite = BytesIO()
    df.to_excel(towrite, index=False, header=True)
    towrite.seek(0)
    b64 = base64.b64encode(towrite.read()).decode()
    href = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="template.xlsx">Excel template'
    return st.markdown(href, unsafe_allow_html=True)

def maximum_drawdowns(price_df):
    """
    Calculate the maximum drawdowns of a dataframe of asset prices.

    Parameters:
    price_df (pd.DataFrame): A pandas DataFrame containing asset prices.
    (date index must be sorted ascending)

    Returns:
    pd.Series: Series of asset names and corresponding maximum drawdowns.
    """
    price_df_sorted = price_df.sort_index(ascending=True)
    max_price_df = price_df_sorted.rolling(window=len(price_df_sorted),min_periods=1).max()
    dd_price_df = price_df_sorted / max_price_df -1
    max_dd_series = dd_price_df.min()

    return max_dd_series

def get_monthly_closing_prices(price_df_daily):
    price_df_monthly = price_df_daily.loc[price_df_daily.groupby(price_df_daily.index.to_period('M')).apply(lambda x: x.index.max())]
    return price_df_monthly


def simulate_leveraged_daily_compounded_annual_return(daily_return,
                                                      daily_vola,
                                                      leverage,
                                                      reference_rate,
                                                      expense_ratio,
                                                      assumed_trading_days,
                                                      sim_runs):
    delta_t = 1/assumed_trading_days
    daily_leverage_cost = ((leverage-1)*reference_rate + expense_ratio)*delta_t

    # run monte carlo simmulation
    daily_return_sim = np.log(1 + leverage*(daily_return + daily_vola*np.random.normal(0, 1, size=(sim_runs, assumed_trading_days))) - daily_leverage_cost)

    daily_compounded_annual_returns = np.sum(daily_return_sim, axis=1)

    mean_daily_compounded_annual_return = daily_compounded_annual_returns.mean()
    std_daily_compounded_annual_return = daily_compounded_annual_returns.std()

    return mean_daily_compounded_annual_return, std_daily_compounded_annual_return

def create_leverage_sim_visual(results_df):
    # Create figure and axis objects
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot mean return on primary y-axis
    ax1.scatter(results_df['Leverage'], results_df['Mean_Return'], label='Simulated return', color='cornflowerblue')
    ax1.set_xlabel('Leverage')
    ax1.set_ylabel('Daily compounded annual return')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))

    plt.grid('on', ls="--")
    # Create secondary y-axis for standard deviation
    ax2 = ax1.twinx()
    ax2.scatter(results_df['Leverage'], results_df['Std_Return'], label='Simulated volatility', color='darkmagenta')
    ax2.set_ylabel('Volatility of annual returns')
    ax2.yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))

    # ask matplotlib for the plotted objects and their labels
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    plt.legend(lines + lines2, labels + labels2, loc=0)

    # Display the plot
    plt.show()

def create_binary_colormap_for_plt_charts(data_values, two_color_list):

    cmap = matplotlib.colors.ListedColormap(two_color_list)

    # scale data
    denominator = max(data_values) - min(data_values)
    scaled_data = [(datum-min(data_values))/denominator for datum in data_values]

    colors = []
    for decimal in scaled_data:
        colors.append(cmap(decimal))

    return colors

def create_colormap_for_plt_charts(data_values, color_list):

    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", color_list)

    # scale data
    denominator = max(data_values) - min(data_values)
    scaled_data = [(datum-min(data_values))/denominator for datum in data_values]

    colors = []
    for decimal in scaled_data:
        colors.append(cmap(decimal))

    return colors

#Technical Analysis functions
def calculate_macd(data, price="Close", days_fast=12, days_slow=26, days_signal=9):
    short_ema = data[price].ewm(span=days_fast, adjust=False).mean()
    long_ema = data[price].ewm(span=days_slow, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=days_signal, adjust=False).mean()
    macd_hist = macd - signal
    return short_ema, long_ema, macd, signal, macd_hist

def pandas_rsi(df: pd.DataFrame, window_length: int = 14, output: str = None, price: str = 'Close'):
    """
    An implementation of Wells Wilder's RSI calculation as outlined in
    his 1978 book "New Concepts in Technical Trading Systems" which makes
    use of the α-1 Wilder Smoothing Method of calculating the average
    gains and losses across trading periods and the Pandas library.

    @author: https://github.com/alphazwest
    Args:
        df: pandas.DataFrame - a Pandas Dataframe object
        window_length: int - the period over which the RSI is calculated. Default is 14
        output: str or None - optional output path to save data as CSV
        price: str - the column name from which the RSI values are calcuated. Default is 'Close'

    Returns:
        DataFrame object with columns as such, where xxx denotes an inconsequential
        name of the provided first column:
            ['xxx', 'diff', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs', 'rsi']
    """
    # Calculate Price Differences using the column specified as price.
    df['diff1'] = df[price].diff(1)

    # Calculate Avg. Gains/Losses
    df['gain'] = df['diff1'].clip(lower=0).round(2)
    df['loss'] = df['diff1'].clip(upper=0).abs().round(2)

    # Get initial Averages
    df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
    df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]

    # Calculate Average Gains
    for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
        df['avg_gain'].iloc[i + window_length + 1] =\
            (df['avg_gain'].iloc[i + window_length] *
             (window_length - 1) +
             df['gain'].iloc[i + window_length + 1])\
            / window_length

    # Calculate Average Losses
    for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
        df['avg_loss'].iloc[i + window_length + 1] =\
            (df['avg_loss'].iloc[i + window_length] *
             (window_length - 1) +
             df['loss'].iloc[i + window_length + 1])\
            / window_length

    # Calculate RS Values
    df['rs'] = df['avg_gain'] / df['avg_loss']

    # Calculate RSI
    df['rsi'] = 100 - (100 / (1.0 + df['rs']))

    # Save if specified
    if output is not None:
        df.to_csv(output)

    return df

def get_cumulatieve_investment_values(returns, payments):
  sum_cum_values = pd.Series(0, index=payments.index, dtype=float)

  for i in payments[payments > 0].index:
    returns_i = returns.loc[i:]
    growth_i = 1 + returns_i
    growth_i[0] = payments[i]
    cum_value_i = growth_i.cumprod()

    sum_cum_values = sum_cum_values.add(cum_value_i, fill_value=0)

  return sum_cum_values


def apply_investment_signal(returns, starting_value, signal):
  date_index = signal.index
  values = pd.Series(starting_value, index=date_index, dtype=float)

  for i in range(1, len(date_index)):
    if signal.iloc[i] == 1:
      values.iloc[i] = values.iloc[i-1] * (1+returns.iloc[i])
    else:
      values.iloc[i] = values.iloc[i-1]

  return values

2024-03-15 22:25:53.174 
════════════════════════════════════════════════
deprecation.showPyplotGlobalUse IS DEPRECATED.
The support for global pyplot instances is planned to be removed soon.

This option will be removed on or after 2024-04-15.

Please update <user defined>.
════════════════════════════════════════════════



In [9]:


tickers = ["AAPL", "XDWD.DE", "ETH-EUR"]



tickers = [x.upper() for x in tickers]
price_df = yf.download(tickers, period='max')["Adj Close"]
price_df = price_df.dropna()

eurusd = yf.download(["EURUSD=X","EUR=X"], period='max')["Adj Close"]


currency = "EUR"



# Currency conversion and long name dictionary creation
curr_conv_tabl = pd.DataFrame(index=price_df.index, columns=price_df.columns)
curr_conv_tabl.fillna(1, inplace=True)

long_name_dict = {}
for col in curr_conv_tabl.columns:
    try:
        long_name_dict[col] = yf.Ticker(col).info["longName"]
        curr = yf.Ticker(col).info["currency"]
        if curr != currency:
            if currency == "EUR":
                curr_conv_tabl[col] = curr_conv_tabl[col] * eurusd["EUR=X"]
            elif currency == "USD":
                curr_conv_tabl[col] = curr_conv_tabl[col] * eurusd["EURUSD=X"]
    except:
        pass

#Paste down last avalible EUR/USD rate (Dec 2003) for datapoints before that date
col_names = list(curr_conv_tabl.columns)
last_exch_rates = curr_conv_tabl.dropna().iloc[0,:].tolist()
last_exch_rate_dict = dict(zip(col_names, last_exch_rates))
curr_conv_tabl.fillna(last_exch_rate_dict, inplace=True)

price_df = price_df * curr_conv_tabl

daily_adjusted_closing_prices = price_df

#Calculate YTD returns
now = price_df.index[-1]
end_prev_y = price_df.index[price_df.index.year<now.year][-1]
ytd_returns = price_df.loc[now] / price_df.loc[end_prev_y] - 1

start_date = daily_adjusted_closing_prices.index.min().date()
end_date = daily_adjusted_closing_prices.index.max().date()

date_range = st.slider("Define the timeframe to be considered for the analysis.", value=[start_date, end_date], min_value=start_date, max_value=end_date, format ='DD.MM.YYYY')
start_date_from_index = daily_adjusted_closing_prices.index[(daily_adjusted_closing_prices.index.day==date_range[0].day) & (daily_adjusted_closing_prices.index.month==date_range[0].month) & (daily_adjusted_closing_prices.index.year==date_range[0].year)].min().date()
end_date_from_index = daily_adjusted_closing_prices.index[(daily_adjusted_closing_prices.index.day==date_range[1].day) & (daily_adjusted_closing_prices.index.month==date_range[1].month) & (daily_adjusted_closing_prices.index.year==date_range[1].year)].max().date()
daily_adjusted_closing_prices = daily_adjusted_closing_prices.loc[start_date_from_index:end_date_from_index]

montly_adjusted_closing_prices = get_monthly_closing_prices(price_df_daily=daily_adjusted_closing_prices)

# get 3-month T-Bill data for Sharpe ratio calculation
UST_3_mo = dr("TB3MS", 'fred',  start=now - datetime.timedelta(days=65))
UST_3_mo.dropna(inplace=True)
UST_3_mo = float(UST_3_mo.iloc[-1])/100

# get SOFR data
SOFR_90_day = dr("SOFR90DAYAVG", 'fred',  start=now - datetime.timedelta(days=10))
SOFR_90_day.dropna(inplace=True)
SOFR_90_day = float(SOFR_90_day.iloc[-1])

  # calculate maximum drawdown
max_dds = maximum_drawdowns(price_df=daily_adjusted_closing_prices)

#montly_adjusted_closing_prices = convert_date_index(montly_adjusted_closing_prices)
monthly_log_returns = np.log(montly_adjusted_closing_prices / montly_adjusted_closing_prices.shift(1))

annualized_mean_returns = monthly_log_returns.mean() * 12
annualized_std_returns = monthly_log_returns.std() * 12**0.5
annualized_cov_returns = monthly_log_returns.cov() * 12
corr_returns = monthly_log_returns.corr()

summary = pd.DataFrame()

summary["mean return"] = annualized_mean_returns
summary["standard deviation"] = annualized_std_returns
summary["Sharpe ratio"] = (summary["mean return"]- UST_3_mo) / summary['standard deviation']
summary["weight"] = 1/len(summary)

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  2 of 2 completed


In [10]:
montly_adjusted_closing_prices

Ticker,AAPL,ETH-EUR,XDWD.DE
Date,,,
2017-11-30,34.2341,375.7892,49.0500
2017-12-29,33.4588,628.2027,49.6300
2018-01-31,31.8549,900.7007,50.2080
2018-02-28,34.5257,701.5578,49.2760
2018-03-29,32.3102,313.7441,47.4680
...,...,...,...
2023-11-30,172.8751,1849.8501,86.3940
2023-12-29,173.7272,2121.0823,89.9600
2024-01-31,169.8406,2162.1553,93.0140


In [15]:
summary.loc["AAPL", "weight"] = 0.5
summary.loc["ETH-EUR", "weight"] = 0.25
summary.loc["XDWD.DE", "weight"] = 0.25
summary

,mean return,standard deviation,Sharpe ratio,weight
Ticker,,,,
AAPL,0.2419,0.2948,0.6427,0.5000
ETH-EUR,0.3481,0.9808,0.3015,0.2500
XDWD.DE,0.1076,0.1434,0.3851,0.2500


In [26]:
test1 = montly_adjusted_closing_prices * summary["weight"]
test1

Ticker,AAPL,ETH-EUR,XDWD.DE
Date,,,
2017-11-30,17.1170,93.9473,12.2625
2017-12-29,16.7294,157.0507,12.4075
2018-01-31,15.9275,225.1752,12.5520
2018-02-28,17.2628,175.3895,12.3190
2018-03-29,16.1551,78.4360,11.8670
...,...,...,...
2023-11-30,86.4376,462.4625,21.5985
2023-12-29,86.8636,530.2706,22.4900
2024-01-31,84.9203,540.5388,23.2535


In [33]:
test1 = daily_adjusted_closing_prices * summary["weight"]
test1 = np.log(test1 / test1.shift(1))
test1.mean()*12

Ticker
AAPL      0.0115
ETH-EUR   0.0194
XDWD.DE   0.0052
dtype: float64

In [34]:
test2 = daily_adjusted_closing_prices
test2 = np.log(test2 / test2.shift(1))
test2 * summary["weight"]
test2.mean()*12

Ticker
AAPL      0.0115
ETH-EUR   0.0194
XDWD.DE   0.0052
dtype: float64

In [ ]:
portfolio_return

In [5]:
market_proxy_input = st.text_input("As per default, the S&P 500 Index (^GSPC) is used as a proxy for the market portfolio. If you consider another index more suitable for your analysis, you can enter its [Yahoo Finace](https://finance.yahoo.com) ticker below (E.g. STOXX Europe 600: ^STOXX, Dax-Performance-Index: ^GDAXI, FTSE 100 Index: ^FTSE)")
riskfree_proxy_input = st.text_input("As per default, 10-year U.S. Treasury yields (^TNX) are used as a proxy for the risk-free rate. You may enter the ticker of a different proxy below (make sure the proxy is quoted in yields, not prices; e.g. 13-week U.S. Treasury yields: ^IRX, 5-year U.S. Treasury yields: ^FVX, 30-year U.S. Treasury yields: ^TYX)")

if market_proxy_input:
    market_proxy = market_proxy_input
else:
    market_proxy = "^GSPC"
if riskfree_proxy_input:
    riskfree_proxy = riskfree_proxy_input
else:
    riskfree_proxy = "^TNX"

proxys_M_rf = [market_proxy, riskfree_proxy]

CAPM_data = yf.download(proxys_M_rf, period='max')["Adj Close"]
CAPM_data.dropna(inplace=True)
CAPM_data = get_monthly_closing_prices(price_df_daily=CAPM_data)

download_sucess2 = False
if len(CAPM_data) < 1:
    st.error("Asset could not be found.")
else:
    if market_proxy_input or riskfree_proxy_input:
        st.success("Proxy updated!")
    download_sucess2 = True

if download_sucess2:
    CAPM_quotes = pd.DataFrame()
    CAPM_quotes[["Market", "risk-free"]] = CAPM_data[proxys_M_rf]
    CAPM_quotes["risk-free"] = CAPM_quotes["risk-free"]/100
    CAPM_quotes = CAPM_quotes.merge(montly_adjusted_closing_prices, left_index=True, right_index=True, how="inner")
    CAPM_returns = np.log(CAPM_quotes / CAPM_quotes.shift(1))
    CAPM_returns["risk-free"] = CAPM_quotes["risk-free"]*(1/12)
    CAPM_returns["MRP"] = CAPM_returns["Market"] - CAPM_returns["risk-free"]

    for asset in monthly_log_returns.columns:
        CAPM_returns[f"{asset}-rf"] = CAPM_returns[asset] - CAPM_returns["risk-free"]

    CAPM_returns.dropna(inplace=True)

    mean_rf = CAPM_returns["risk-free"].mean()*12
    mean_MRP = CAPM_returns["MRP"].mean()*12

    CAPM_summary = pd.DataFrame()
    for asset in monthly_log_returns.columns:
        Y=CAPM_returns[f"{asset}-rf"]
        X= sm.add_constant(CAPM_returns["MRP"])
        regress = sm.OLS(Y, X)
        result = regress.fit()
        CAPM_summary.loc[asset,"Mean rf"] = mean_rf
        CAPM_summary.loc[asset,"Beta"] = float(result.params[1])
        CAPM_summary.loc[asset,"Mean MRP"] = mean_MRP
        CAPM_summary.loc[asset,"Fair return"] = CAPM_summary.loc[asset,"Mean rf"] + CAPM_summary.loc[asset,"Beta"]*CAPM_summary.loc[asset,"Mean MRP"]
        CAPM_summary.loc[asset,"Alpha"] = float(result.params[0]*12)
        CAPM_summary.loc[asset,"Mean return"] = CAPM_summary.loc[asset,"Fair return"] + CAPM_summary.loc[asset,"Alpha"]

CAPM_summary

[*********************100%%**********************]  2 of 2 completed


,Mean rf,Beta,Mean MRP,Fair return,Alpha,Mean return
AAPL,0.0248,1.0634,0.0851,0.1152,0.1401,0.2553
ETH-EUR,0.0248,2.7085,0.0851,0.2551,0.1113,0.3664
XDWD.DE,0.0248,0.7166,0.0851,0.0857,0.0279,0.1136


In [41]:
def run_CAPM(price_df_monthly, CAPM_data, proxys_M_rf):
  CAPM_quotes = pd.DataFrame()
  CAPM_quotes[["Market", "risk-free"]] = CAPM_data[proxys_M_rf]
  CAPM_quotes["risk-free"] = CAPM_quotes["risk-free"]/100
  CAPM_quotes = CAPM_quotes.merge(price_df_monthly, left_index=True, right_index=True, how="inner")
  CAPM_returns = np.log(CAPM_quotes / CAPM_quotes.shift(1))
  CAPM_returns["risk-free"] = CAPM_quotes["risk-free"]*(1/12)
  CAPM_returns["MRP"] = CAPM_returns["Market"] - CAPM_returns["risk-free"]

  for asset in monthly_log_returns.columns:
      CAPM_returns[f"{asset}-rf"] = CAPM_returns[asset] - CAPM_returns["risk-free"]

  CAPM_returns.dropna(inplace=True)

  mean_rf = CAPM_returns["risk-free"].mean()*12
  mean_MRP = CAPM_returns["MRP"].mean()*12

  CAPM_summary = pd.DataFrame()
  for asset in monthly_log_returns.columns:
      Y=CAPM_returns[f"{asset}-rf"]
      X= sm.add_constant(CAPM_returns["MRP"])
      regress = sm.OLS(Y, X)
      result = regress.fit()
      CAPM_summary.loc[asset,"Mean rf"] = mean_rf
      CAPM_summary.loc[asset,"Beta"] = float(result.params[1])
      CAPM_summary.loc[asset,"Mean MRP"] = mean_MRP
      CAPM_summary.loc[asset,"Fair return"] = CAPM_summary.loc[asset,"Mean rf"] + CAPM_summary.loc[asset,"Beta"]*CAPM_summary.loc[asset,"Mean MRP"]
      CAPM_summary.loc[asset,"Alpha"] = float(result.params[0]*12)
      CAPM_summary.loc[asset,"Mean return"] = CAPM_summary.loc[asset,"Fair return"] + CAPM_summary.loc[asset,"Alpha"]

  return CAPM_summary, mean_rf, mean_MRP

In [43]:
test = run_CAPM(montly_adjusted_closing_prices, CAPM_data, proxys_M_rf)[0]
test2 = run_CAPM(montly_adjusted_closing_prices, CAPM_data, proxys_M_rf)[1]
test2

0.024770416609115067